# Preparação dos dados
Nesse notebook vamos preparar os dados que utilizaremos ao longo do desenvolvimento do projeto. Os datasets aqui utilizados são importados do portal de dados abertos da UFRN (http://dados.ufrn.br/).

Para esse estudo, analisaremos apenas os dados do curso de Ciência e Tecnologia da UFRN, mais espeficamente a disciplina de Cálculo 2, onde tentaremos prever as possiveis reprovações dos alunos se baseando em seu histórico e na nota da primeira unidade na disciplina.

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importando os dados
Vamos importar e já separar os escolhidos.

In [2]:
lista_anos_analisados = ['20151', '20152', '20161', '20162', '20171', '20172'] # 20172 vai ficar para validação

matriculas_geral = pd.DataFrame()
for ano in lista_anos_analisados:
    matriculas = pd.read_csv(f"data/matriculas_pos_limpeza_{ano}.csv", sep=";")
    
    matriculas_geral = pd.concat([matriculas_geral, matriculas])

/home/rafael/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [3]:
matriculas_geral.shape

(34473, 12)

In [4]:
matriculas_geral.head()

,Unnamed: 0,Unnamed: 10,descricao,discente,faltas_unidade,id_curso,id_turma,media_final,nota,numero_total_faltas,reposicao,unidade
0,34925,NaN,REPROVADO,6c4acceb9b1288b61d893dc1b5005764,0.0,10320810.0,1242602,0.0,0.0,0.0,f,1.0
1,34926,NaN,REPROVADO,6c4acceb9b1288b61d893dc1b5005764,0.0,10320810.0,1242602,0.0,0.0,0.0,f,2.0
2,34927,NaN,REPROVADO,6c4acceb9b1288b61d893dc1b5005764,0.0,10320810.0,1242602,0.0,0.0,0.0,f,3.0
3,34928,NaN,APROVADO,bb67c2ca87d0e1fcf027b2f0f6df4221,0.0,10320810.0,1242602,8.5,9.0,0.0,f,2.0
4,34929,NaN,APROVADO,bb67c2ca87d0e1fcf027b2f0f6df4221,0.0,10320810.0,1242602,8.5,10.0,0.0,f,3.0


In [5]:
matriculas_geral.columns

Index(['Unnamed: 0', 'Unnamed: 10', 'descricao', 'discente', 'faltas_unidade',
       'id_curso', 'id_turma', 'media_final', 'nota', 'numero_total_faltas',
       'reposicao', 'unidade'],
      dtype='object')

In [6]:
matriculas_geral.descricao.value_counts()

REPROVADO                           14079
APROVADO POR NOTA                    9975
APROVADO                             5664
REPROVADO POR MÉDIA E POR FALTAS     4704
REPROVADO POR NOTA                     48
REPROVADO POR FALTAS                    3
Name: descricao, dtype: int64

In [7]:
status_aprovado = ['APROVADO', 'APROVADO POR NOTA']

matriculas_geral['reprovou'] = matriculas_geral.descricao.apply(lambda x: False if x in status_aprovado else True)

In [8]:
columns_matriculas = ['discente', 'faltas_unidade',
       'id_turma', 'media_final', 'nota', 'numero_total_faltas',
       'reposicao', 'unidade', 'reprovou']
matriculas_geral = matriculas_geral[columns_matriculas]

In [9]:
matriculas_geral.to_csv("data/matriculas_geral_pos_preparacao.csv", sep=';', index=False)

### Importando os dados de turmas

In [10]:
turmas_geral = pd.DataFrame()
for ano in lista_anos_analisados:
    turmas = pd.read_csv(f"data/turmas_pos_limpeza_{ano}.csv", sep=";")
    
    turmas_geral = pd.concat([turmas_geral, turmas])

In [11]:
turmas_geral.shape

(140, 6)

In [12]:
turmas_geral.head()

,Unnamed: 0,id_turma,id_componente_curricular,id_docente_interno,ano_periodo,situacao_turma
0,5294,1242221,57586,5752289.0,20151,CONSOLIDADA
1,5297,1242224,57586,5755052.0,20151,CONSOLIDADA
2,5301,1242227,57586,5755040.0,20151,CONSOLIDADA
3,5304,1242230,57586,5757117.0,20151,CONSOLIDADA
4,5306,1242232,57586,5756876.0,20151,CONSOLIDADA


In [13]:
turmas_geral.columns

Index(['Unnamed: 0', 'id_turma', 'id_componente_curricular',
       'id_docente_interno', 'ano_periodo', 'situacao_turma'],
      dtype='object')

In [14]:
columns_turma = ['id_turma', 'id_componente_curricular','id_docente_interno', 'ano_periodo']
turmas_geral = turmas_geral[columns_turma]

In [15]:
turmas_geral.to_csv("data/turmas_geral_pos_preparacao.csv", sep=';', index=False)

## Pegando apenas os dados de Cálculo II

In [16]:
id_componente_c2 = ['48584', '57588', '2051052']
turmas_c2 = turmas_geral[turmas_geral.id_componente_curricular.isin(id_componente_c2)]

In [17]:
# Pegando apenas dos periodos que seram analisados: 2016.1, 2016.2, 2017.1
turmas_c2 = turmas_c2[turmas_c2.ano_periodo.isin(['20161', '20162', '20171'])]

In [18]:
turmas_c2.id_docente_interno.value_counts()

5752242.0    3
5752287.0    2
5752769.0    2
5758125.0    2
5756827.0    2
5757166.0    1
5753045.0    1
5753037.0    1
5757117.0    1
5758644.0    1
Name: id_docente_interno, dtype: int64

In [19]:
turmas_geral.id_docente_interno.value_counts()

26592.0      12
5757193.0    11
5752240.0    10
5752289.0     8
5752287.0     8
5753020.0     7
5757117.0     7
5758125.0     7
5753045.0     7
5752242.0     6
5757166.0     5
5755052.0     4
5756620.0     4
5758715.0     4
5753037.0     4
5753081.0     4
5759223.0     4
5752170.0     4
5756966.0     3
5758220.0     3
5752769.0     3
5752727.0     3
5756827.0     3
5756876.0     2
5757983.0     2
5757657.0     1
5756199.0     1
5755040.0     1
5758644.0     1
5756031.0     1
Name: id_docente_interno, dtype: int64

In [24]:
matriculas_geral[matriculas_geral.id_turma.isin(turmas_c2.id_turma.unique())]

,discente,faltas_unidade,id_turma,media_final,nota,numero_total_faltas,reposicao,unidade,reprovou
684,324ebf7db0ca091e704046c9da1b7338,0.0,57573583,6.2,6.0,2.0,f,1.0,False
685,324ebf7db0ca091e704046c9da1b7338,0.0,57573583,6.2,6.0,2.0,f,2.0,False
686,324ebf7db0ca091e704046c9da1b7338,0.0,57573583,6.2,6.5,2.0,f,3.0,False
687,a3bc6f821ed792af2d8d67f97c4ddb5b,0.0,57573583,5.7,3.0,0.0,f,1.0,False
688,a3bc6f821ed792af2d8d67f97c4ddb5b,0.0,57573583,5.7,6.5,0.0,f,2.0,False
689,a3bc6f821ed792af2d8d67f97c4ddb5b,0.0,57573583,5.7,7.5,0.0,f,3.0,False
690,25dd0216691e1644cfb19c0bfb7ff8b3,0.0,57573583,5.5,3.0,0.0,f,1.0,False
691,25dd0216691e1644cfb19c0bfb7ff8b3,0.0,57573583,5.5,6.0,0.0,f,2.0,False
692,25dd0216691e1644cfb19c0bfb7ff8b3,0.0,57573583,5.5,7.5,0.0,f,3.0,False
693,621fa53cd5bf23d9d8b300354f86fa4b,0.0,57573583,5.0,4.5,0.0,f,2.0,False
